In [1]:
import pymc as pm
import arviz as az
import pytensor
import pytensor.tensor as pt
import numpy as np

import sys

sys.path.append("../..")
from cge_modeling import CGEModel
from cge_modeling.pytensorf.optimize import root
from cge_modeling.pytensorf.compile import compile_cge_model_to_pytensor_Op
from tests.utilities.models import load_model_1

In [2]:
mod = load_model_1(backend="pytensor", parse_equations_to_sympy=False)

In [3]:
obs_data = np.array([11000, 11000, 7000, np.nan, 1, 1, 0])
not_na = ~np.isnan(obs_data)
eye = np.eye(7)
Z = eye[not_na, :]
not_Z = eye[~not_na, :]

In [4]:
def predict_unobserved(Z, not_Z, mu, cov, y_obs):
    mu_o = Z @ mu
    mu_u = not_Z @ mu
    cov_oo = Z @ cov @ Z.T
    cov_uu = not_Z @ cov @ not_Z.T
    cov_uo = not_Z @ cov @ Z.T

    cov_oo_inv = pt.linalg.solve(cov_oo, pt.identity_like(cov_oo), assume_a="pos")

    beta = cov_uo @ cov_oo_inv
    resid = y_obs - mu_o

    mu_hat = mu_u + beta @ resid
    Sigma_hat = cov_uu - beta @ cov_oo @ beta.T

    return mu_hat, Sigma_hat

In [25]:
coords = {
    "equation": np.arange(7, dtype=int),
    "equation_aux": np.arange(7, dtype=int),
    "variable": mod.variable_names,
    "obs_variable": [name for name, observed in zip(mod.variable_names, not_na) if observed],
    "missing_variable": [
        name for name, observed in zip(mod.variable_names, not_na) if not observed
    ],
    "parameter": mod.parameter_names,
}

with pm.Model(coords=coords) as m:
    # Parameters
    pm_A = pm.Gamma("A", 2, 1)
    pm_alpha = pm.Beta("alpha", 3, 3)
    pm_K_s = pm.Normal("K_s", 4000, 100)
    pm_L_s = pm.Normal("L_s", 7000, 100)

    Y0 = pm.Normal("Y", 11000, 1000)
    C0 = pm.Normal("C", 11000, 1000)
    Kd0 = pm.Normal("K_d", 4000, 100)
    Ld0 = pm.Normal("L_d", 7000, 100)
    income0 = pm.Normal("income", 11000, 1000)

    input_dict = {
        "Y": Y0,
        "C": C0,
        "K_d": Kd0,
        "L_d": Ld0,
        "r": pt.as_tensor_variable(1.0),
        "P": pt.as_tensor_variable(1.0),
        "resid": pt.as_tensor_variable(0.0),
        "income": income0,
        "A": pm_A,
        "alpha": pm_alpha,
        "K_s": pm_K_s,
        "L_s": pm_L_s,
        "w": pt.as_tensor_variable(1.0),
    }

    x0_dict = {k.name: input_dict[k.name].eval() for k in mod.variables}
    param_dict = {k.name: input_dict[k.name].eval() for k in mod.parameters}

    fake_data = mod.generate_SAM_from_parameters(param_dict, x0_dict)

#     f_model, f_jac, f_jac_inv = compile_cge_model_to_pytensor_Op(mod, inverse_method='solve')

#     root_histories, converged, step_size, n_steps = root(f_model, f_jac_inv,
#                                                          initial_data=x0_dict,
#                                                          parameters=param_dict)

#     solution = [history[-1] for history in root_histories]
#     flat_solution = pt.concatenate([pt.atleast_1d(x).ravel() for x in solution])

#     error = pm.Deterministic("error", f_model(*solution, *list(param_dict.values())), dims=["equation"])
#     success = pm.Deterministic("success", pt.allclose(error, 0))

#     solution_jac = f_jac(*solution, *list(param_dict.values()))
#     ls = pm.Gamma('length_scale', 2, 1)
#     sigma = pm.HalfNormal("sigma", [10, 10, 10, 10, 0.1, 0.1, 0.1], dims=["variable"])
#     cov = sigma ** 2 * pm.gp.cov.Matern32(solution_jac, ls=ls, active_dims=7)

#     y_hat = pm.MvNormal(
#         "y_hat", mu=Z @ flat_solution, cov=Z @ K @ Z.T, dims=["obs_variable"], observed=obs_data[not_na]
#     )

#         missing_mu, missing_cov = predict_unobserved(Z, not_Z, solution, cov, obs_data[not_na])
#     y_missing = pm.MvNormal('y_missing', mu=missing_mu, cov=missing_cov, dims=['missing_variable'])

#     pm.Potential("optimizer_failure", pt.switch(success, 0, -np.inf))

#     idata = pm.sample(nuts_sampler="numpyro")
#     idata = pm.sample_prior_predictive()
#     idata = pm.sample_smc(kernel=pm.smc.MH,
#                           draws=5_000,
#                           correlation_threshold=1e-2,
#                           threshold=0.5,
#                           chains=4,

#                           progressbar=False)

In [26]:
fake_data

{'Y': array(3654.52091352),
 'L_d': array(6895.3077747),
 'K_d': array(4074.69001939),
 'C': array(3654.52091352),
 'income': array(13294.54667584),
 'r': array(1.57048484),
 'P': array(3.63783571),
 'resid': array(9.79790665e-09)}

In [7]:
var_names = ["~root", "~params", "~x0", "~error", "~success"]
az.summary(idata, var_names=var_names)

NameError: name 'idata' is not defined